# Student Alcohol Consumption Analysis

## Project Overview
This comprehensive analysis explores student alcohol consumption patterns and their relationship with academic performance, demographics, and social factors.

**Key Research Questions:**
1. How does alcohol consumption vary across different demographic groups?
2. What factors most strongly correlate with alcohol consumption?
3. Can we predict academic performance based on alcohol consumption patterns?
4. What's the relationship between weekday vs weekend drinking patterns?

**Dataset:** Portuguese student performance data with 33 features including demographics, family background, school-related features, and alcohol consumption levels.

## 1. Import Libraries and Setup

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
from scipy import stats

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Libraries imported successfully!")

## 2. Data Loading and Initial Exploration

In [ ]:
# Load the dataset
# Note: Make sure to download the student-mat.csv file from the UCI repository
# URL: https://archive.ics.uci.edu/ml/datasets/Student+Performance

try:
    df = pd.read_csv('student-mat.csv', sep=';')
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print("Dataset file not found. Please download 'student-mat.csv' from UCI repository.")
    print("URL: https://archive.ics.uci.edu/ml/datasets/Student+Performance")
    # For demonstration, we'll create a sample dataset structure
    print("Creating sample dataset for demonstration...")
    
# Display basic information
print(f"\nDataset shape: {df.shape}")
print(f"\nColumn names: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Comprehensive data overview
print("=== DATASET OVERVIEW ===")
print(f"Number of students: {len(df)}")
print(f"Number of features: {len(df.columns)}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")

print("\n=== DATA TYPES ===")
print(df.dtypes.value_counts())

print("\n=== MISSING VALUES ===")
missing_data = df.isnull().sum()
if missing_data.sum() == 0:
    print("No missing values found!")
else:
    print(missing_data[missing_data > 0])

print("\n=== BASIC STATISTICS ===")
df.describe()

## 3. Comprehensive Data Preprocessing

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

# 1. Handle categorical variables consistently
print("=== PREPROCESSING STEPS ===")

# Binary categorical variables (yes/no)
binary_cols = ['schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
for col in binary_cols:
    if col in df_processed.columns:
        df_processed[col] = df_processed[col].map({'yes': 1, 'no': 0})

# Ordinal categorical variables
ordinal_mappings = {
    'Medu': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4},  # Mother's education
    'Fedu': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4},  # Father's education
    'traveltime': {1: 1, 2: 2, 3: 3, 4: 4},  # Travel time
    'studytime': {1: 1, 2: 2, 3: 3, 4: 4},   # Study time
    'famrel': {1: 1, 2: 2, 3: 3, 4: 4, 5: 5}, # Family relationship
    'freetime': {1: 1, 2: 2, 3: 3, 4: 4, 5: 5}, # Free time
    'goout': {1: 1, 2: 2, 3: 3, 4: 4, 5: 5},    # Going out
    'Dalc': {1: 1, 2: 2, 3: 3, 4: 4, 5: 5},     # Workday alcohol
    'Walc': {1: 1, 2: 2, 3: 3, 4: 4, 5: 5},     # Weekend alcohol
    'health': {1: 1, 2: 2, 3: 3, 4: 4, 5: 5}    # Health status
}

# Apply ordinal mappings (if needed)
for col, mapping in ordinal_mappings.items():
    if col in df_processed.columns:
        df_processed[col] = df_processed[col].map(mapping).fillna(df_processed[col])

# 2. Create meaningful features
print("Creating derived features...")

# Alcohol consumption features
df_processed['total_alcohol'] = df_processed['Dalc'] + df_processed['Walc']
df_processed['alcohol_ratio'] = df_processed['Walc'] / (df_processed['Dalc'] + 0.1)  # Avoid division by zero
df_processed['high_alcohol'] = ((df_processed['Dalc'] >= 3) | (df_processed['Walc'] >= 3)).astype(int)
df_processed['weekend_drinker'] = (df_processed['Walc'] > df_processed['Dalc']).astype(int)

# Academic performance features
df_processed['grade_improvement'] = df_processed['G3'] - df_processed['G1']
df_processed['grade_consistency'] = df_processed[['G1', 'G2', 'G3']].std(axis=1)
df_processed['avg_grade'] = df_processed[['G1', 'G2', 'G3']].mean(axis=1)

# Family background features
df_processed['parents_edu_avg'] = (df_processed['Medu'] + df_processed['Fedu']) / 2
df_processed['edu_gap'] = abs(df_processed['Medu'] - df_processed['Fedu'])

# Social features
df_processed['social_score'] = df_processed['freetime'] + df_processed['goout']
df_processed['support_score'] = df_processed['schoolsup'] + df_processed['famsup']

print(f"Features created. New shape: {df_processed.shape}")

# 3. Handle nominal categorical variables with one-hot encoding
nominal_cols = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian']
existing_nominal = [col for col in nominal_cols if col in df_processed.columns]

if existing_nominal:
    print(f"One-hot encoding: {existing_nominal}")
    df_processed = pd.get_dummies(df_processed, columns=existing_nominal, drop_first=True)

print(f"Final processed shape: {df_processed.shape}")
print("Preprocessing completed successfully!")

## 4. Alcohol Consumption Analysis

In [ ]:
# Comprehensive alcohol consumption analysis
print("=== ALCOHOL CONSUMPTION PATTERNS ===")

# Basic statistics
print("\n1. Basic Alcohol Consumption Statistics:")
alcohol_stats = df[['Dalc', 'Walc']].describe()
print(alcohol_stats)

print(f"\nWeekday vs Weekend Consumption:")
print(f"Average weekday consumption: {df['Dalc'].mean():.2f}")
print(f"Average weekend consumption: {df['Walc'].mean():.2f}")
print(f"Students with higher weekend consumption: {(df['Walc'] > df['Dalc']).sum()} ({(df['Walc'] > df['Dalc']).mean()*100:.1f}%)")

# Distribution analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Weekday alcohol distribution
axes[0,0].hist(df['Dalc'], bins=5, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('Weekday Alcohol Consumption Distribution')
axes[0,0].set_xlabel('Dalc (1=very low, 5=very high)')
axes[0,0].set_ylabel('Number of Students')

# Weekend alcohol distribution
axes[0,1].hist(df['Walc'], bins=5, alpha=0.7, color='lightcoral', edgecolor='black')
axes[0,1].set_title('Weekend Alcohol Consumption Distribution')
axes[0,1].set_xlabel('Walc (1=very low, 5=very high)')
axes[0,1].set_ylabel('Number of Students')

# Comparison boxplot
alcohol_data = pd.melt(df[['Dalc', 'Walc']], var_name='Day_Type', value_name='Consumption')
sns.boxplot(data=alcohol_data, x='Day_Type', y='Consumption', ax=axes[1,0])
axes[1,0].set_title('Weekday vs Weekend Alcohol Consumption')
axes[1,0].set_xlabel('Day Type')
axes[1,0].set_ylabel('Alcohol Consumption Level')

# Correlation between Dalc and Walc
axes[1,1].scatter(df['Dalc'], df['Walc'], alpha=0.6, color='purple')
axes[1,1].set_title(f'Weekday vs Weekend Alcohol Correlation\nr = {df["Dalc"].corr(df["Walc"]):.3f}')
axes[1,1].set_xlabel('Weekday Alcohol (Dalc)')
axes[1,1].set_ylabel('Weekend Alcohol (Walc)')

plt.tight_layout()
plt.show()

# Statistical test for difference between weekday and weekend consumption
t_stat, p_value = stats.ttest_rel(df['Dalc'], df['Walc'])
print(f"\n2. Statistical Test (Paired t-test):")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Significant difference: {'Yes' if p_value < 0.05 else 'No'}")

In [ ]:
# Alcohol consumption by demographics
print("=== ALCOHOL CONSUMPTION BY DEMOGRAPHICS ===")

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# By Gender
df_gender = df.groupby('sex')[['Dalc', 'Walc']].mean()
df_gender.plot(kind='bar', ax=axes[0,0], color=['skyblue', 'lightcoral'])
axes[0,0].set_title('Average Alcohol Consumption by Gender')
axes[0,0].set_ylabel('Average Consumption')
axes[0,0].legend(['Weekday', 'Weekend'])
axes[0,0].tick_params(axis='x', rotation=0)

# By Age
df_age = df.groupby('age')[['Dalc', 'Walc']].mean()
df_age.plot(kind='bar', ax=axes[0,1], color=['skyblue', 'lightcoral'])
axes[0,1].set_title('Average Alcohol Consumption by Age')
axes[0,1].set_ylabel('Average Consumption')
axes[0,1].legend(['Weekday', 'Weekend'])
axes[0,1].tick_params(axis='x', rotation=45)

# By Family Size
if 'famsize' in df.columns:
    df_famsize = df.groupby('famsize')[['Dalc', 'Walc']].mean()
    df_famsize.plot(kind='bar', ax=axes[0,2], color=['skyblue', 'lightcoral'])
    axes[0,2].set_title('Average Alcohol Consumption by Family Size')
    axes[0,2].set_ylabel('Average Consumption')
    axes[0,2].legend(['Weekday', 'Weekend'])
    axes[0,2].tick_params(axis='x', rotation=0)

# By Going Out Frequency
df_goout = df.groupby('goout')[['Dalc', 'Walc']].mean()
df_goout.plot(kind='bar', ax=axes[1,0], color=['skyblue', 'lightcoral'])
axes[1,0].set_title('Alcohol Consumption by Going Out Frequency')
axes[1,0].set_ylabel('Average Consumption')
axes[1,0].legend(['Weekday', 'Weekend'])
axes[1,0].tick_params(axis='x', rotation=0)

# By Romantic Relationship
df_romantic = df.groupby('romantic')[['Dalc', 'Walc']].mean()
df_romantic.plot(kind='bar', ax=axes[1,1], color=['skyblue', 'lightcoral'])
axes[1,1].set_title('Alcohol Consumption by Romantic Relationship')
axes[1,1].set_ylabel('Average Consumption')
axes[1,1].legend(['Weekday', 'Weekend'])
axes[1,1].tick_params(axis='x', rotation=0)

# By Free Time
df_freetime = df.groupby('freetime')[['Dalc', 'Walc']].mean()
df_freetime.plot(kind='bar', ax=axes[1,2], color=['skyblue', 'lightcoral'])
axes[1,2].set_title('Alcohol Consumption by Free Time')
axes[1,2].set_ylabel('Average Consumption')
axes[1,2].legend(['Weekday', 'Weekend'])
axes[1,2].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

# Print some key insights
print("\n3. Key Demographic Insights:")
print(f"Gender differences:")
for gender in df['sex'].unique():
    subset = df[df['sex'] == gender]
    print(f"  {gender}: Weekday = {subset['Dalc'].mean():.2f}, Weekend = {subset['Walc'].mean():.2f}")

print(f"\nAge-related patterns:")
age_corr_dalc = df['age'].corr(df['Dalc'])
age_corr_walc = df['age'].corr(df['Walc'])
print(f"  Age correlation with weekday drinking: {age_corr_dalc:.3f}")
print(f"  Age correlation with weekend drinking: {age_corr_walc:.3f}")

## 5. Academic Performance vs Alcohol Consumption

In [ ]:
# Academic performance analysis
print("=== ACADEMIC PERFORMANCE vs ALCOHOL CONSUMPTION ===")

# Calculate correlations
performance_cols = ['G1', 'G2', 'G3', 'absences', 'studytime', 'failures']
alcohol_cols = ['Dalc', 'Walc']

print("\n1. Correlation Analysis:")
correlation_matrix = df[performance_cols + alcohol_cols].corr()
print("\nCorrelations with Alcohol Consumption:")
for perf_col in performance_cols:
    dalc_corr = df[perf_col].corr(df['Dalc'])
    walc_corr = df[perf_col].corr(df['Walc'])
    print(f"  {perf_col}: Dalc = {dalc_corr:.3f}, Walc = {walc_corr:.3f}")

# Visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Grade distributions by alcohol consumption levels
# High vs Low alcohol consumption
df['high_dalc'] = (df['Dalc'] >= 3).astype(int)
df['high_walc'] = (df['Walc'] >= 3).astype(int)

# G3 by weekday alcohol
df.boxplot(column='G3', by='high_dalc', ax=axes[0,0])
axes[0,0].set_title('Final Grade (G3) by Weekday Alcohol Consumption')
axes[0,0].set_xlabel('High Weekday Alcohol (0=Low, 1=High)')
axes[0,0].set_ylabel('Final Grade (G3)')

# G3 by weekend alcohol
df.boxplot(column='G3', by='high_walc', ax=axes[0,1])
axes[0,1].set_title('Final Grade (G3) by Weekend Alcohol Consumption')
axes[0,1].set_xlabel('High Weekend Alcohol (0=Low, 1=High)')
axes[0,1].set_ylabel('Final Grade (G3)')

# Scatter plot: G3 vs Total Alcohol
total_alcohol = df['Dalc'] + df['Walc']
axes[0,2].scatter(total_alcohol, df['G3'], alpha=0.6, color='red')
axes[0,2].set_title(f'Final Grade vs Total Alcohol Consumption\nr = {total_alcohol.corr(df["G3"]):.3f}')
axes[0,2].set_xlabel('Total Alcohol Consumption (Dalc + Walc)')
axes[0,2].set_ylabel('Final Grade (G3)')

# Absences by alcohol consumption
df.boxplot(column='absences', by='high_dalc', ax=axes[1,0])
axes[1,0].set_title('Absences by Weekday Alcohol Consumption')
axes[1,0].set_xlabel('High Weekday Alcohol (0=Low, 1=High)')
axes[1,0].set_ylabel('Number of Absences')

# Study time by alcohol consumption
df.boxplot(column='studytime', by='high_walc', ax=axes[1,1])
axes[1,1].set_title('Study Time by Weekend Alcohol Consumption')
axes[1,1].set_xlabel('High Weekend Alcohol (0=Low, 1=High)')
axes[1,1].set_ylabel('Study Time')

# Failures by alcohol consumption
df.boxplot(column='failures', by='high_walc', ax=axes[1,2])
axes[1,2].set_title('Past Failures by Weekend Alcohol Consumption')
axes[1,2].set_xlabel('High Weekend Alcohol (0=Low, 1=High)')
axes[1,2].set_ylabel('Number of Past Failures')

plt.tight_layout()
plt.show()

# Statistical tests
print("\n2. Statistical Tests:")
# T-test for grades between high and low alcohol consumers
low_dalc_grades = df[df['high_dalc'] == 0]['G3']
high_dalc_grades = df[df['high_dalc'] == 1]['G3']
t_stat, p_val = stats.ttest_ind(low_dalc_grades, high_dalc_grades)
print(f"Weekday alcohol impact on G3: t={t_stat:.3f}, p={p_val:.4f}")

low_walc_grades = df[df['high_walc'] == 0]['G3']
high_walc_grades = df[df['high_walc'] == 1]['G3']
t_stat, p_val = stats.ttest_ind(low_walc_grades, high_walc_grades)
print(f"Weekend alcohol impact on G3: t={t_stat:.3f}, p={p_val:.4f}")

# Group analysis
print("\n3. Group Performance Analysis:")
print("Average G3 by alcohol consumption groups:")
print(f"  Low weekday alcohol: {low_dalc_grades.mean():.2f}")
print(f"  High weekday alcohol: {high_dalc_grades.mean():.2f}")
print(f"  Low weekend alcohol: {low_walc_grades.mean():.2f}")
print(f"  High weekend alcohol: {high_walc_grades.mean():.2f}")

## 6. Advanced Feature Engineering

In [ ]:
# Advanced feature engineering for better predictions
print("=== ADVANCED FEATURE ENGINEERING ===")

# Use the processed dataframe
df_features = df_processed.copy()

# Create interaction features
print("Creating interaction features...")
df_features['alcohol_x_goout'] = df_features['total_alcohol'] * df_features['goout']
df_features['alcohol_x_freetime'] = df_features['total_alcohol'] * df_features['freetime']
df_features['studytime_x_failures'] = df_features['studytime'] * df_features['failures']
df_features['absences_x_alcohol'] = df_features['absences'] * df_features['total_alcohol']

# Create polynomial features for key variables
df_features['studytime_squared'] = df_features['studytime'] ** 2
df_features['absences_squared'] = df_features['absences'] ** 2
df_features['total_alcohol_squared'] = df_features['total_alcohol'] ** 2

# Binning features
df_features['age_group'] = pd.cut(df_features['age'], bins=[14, 16, 18, 22], labels=['Young', 'Middle', 'Old'])
df_features['grade_category'] = pd.cut(df_features['avg_grade'], bins=[0, 10, 15, 20], labels=['Low', 'Medium', 'High'])

# One-hot encode the new categorical features
df_features = pd.get_dummies(df_features, columns=['age_group', 'grade_category'], drop_first=True)

print(f"Feature engineering completed. New shape: {df_features.shape}")

# Feature importance analysis using correlation
print("\nAnalyzing feature importance...")
numeric_cols = df_features.select_dtypes(include=[np.number]).columns
target_correlations = df_features[numeric_cols].corr()['G3'].abs().sort_values(ascending=False)

print("\nTop 15 features correlated with G3:")
for i, (feature, corr) in enumerate(target_correlations.head(15).items()):
    if feature != 'G3':
        print(f"{i+1:2d}. {feature:25s}: {corr:.3f}")

# Visualize top correlations
plt.figure(figsize=(12, 8))
top_features = target_correlations.head(16)[1:]  # Exclude G3 itself
plt.barh(range(len(top_features)), top_features.values)
plt.yticks(range(len(top_features)), top_features.index)
plt.xlabel('Absolute Correlation with G3')
plt.title('Top 15 Features Correlated with Final Grade (G3)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Select best features for modeling
feature_cols = [col for col in df_features.columns if col not in ['G1', 'G2', 'G3'] and col in numeric_cols]
X = df_features[feature_cols]
y = df_features['G3']

print(f"\nSelected {len(feature_cols)} features for modeling.")
print(f"Target variable (G3) statistics:")
print(f"  Mean: {y.mean():.2f}")
print(f"  Std: {y.std():.2f}")
print(f"  Range: {y.min():.1f} - {y.max():.1f}")

## 7. Machine Learning Models with Hyperparameter Tuning

In [ ]:
# Comprehensive machine learning pipeline
print("=== MACHINE LEARNING PIPELINE ===")

# Handle any remaining NaN values
X = X.fillna(X.mean())
y = y.fillna(y.mean())

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=None)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

# Define models with hyperparameters
models = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {},
        'scaled': False
    },
    'Ridge Regression': {
        'model': Ridge(),
        'params': {'alpha': [0.1, 1.0, 10.0, 100.0]},
        'scaled': True
    },
    'Lasso Regression': {
        'model': Lasso(),
        'params': {'alpha': [0.01, 0.1, 1.0, 10.0]},
        'scaled': True
    },
    'Random Forest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10]
        },
        'scaled': False
    },
    'Gradient Boosting': {
        'model': GradientBoostingRegressor(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7]
        },
        'scaled': False
    },
    'Support Vector Regression': {
        'model': SVR(),
        'params': {
            'C': [0.1, 1.0, 10.0],
            'gamma': ['scale', 'auto'],
            'kernel': ['rbf', 'linear']
        },
        'scaled': True
    }
}

# Train and evaluate models
results = {}
cv = KFold(n_splits=5, shuffle=True, random_state=42)

print("\nTraining and evaluating models...")
for name, model_config in models.items():
    print(f"\nTraining {name}...")
    
    # Select appropriate data (scaled or unscaled)
    if model_config['scaled']:
        X_train_use = X_train_scaled
        X_test_use = X_test_scaled
    else:
        X_train_use = X_train
        X_test_use = X_test
    
    # Hyperparameter tuning
    if model_config['params']:
        grid_search = GridSearchCV(
            model_config['model'],
            model_config['params'],
            cv=cv,
            scoring='r2',
            n_jobs=-1
        )
        grid_search.fit(X_train_use, y_train)
        best_model = grid_search.best_estimator_
        print(f"  Best parameters: {grid_search.best_params_}")
    else:
        best_model = model_config['model']
        best_model.fit(X_train_use, y_train)
    
    # Cross-validation scores
    cv_scores = cross_val_score(best_model, X_train_use, y_train, cv=cv, scoring='r2')
    
    # Test set predictions
    y_pred = best_model.predict(X_test_use)
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'model': best_model,
        'cv_r2_mean': cv_scores.mean(),
        'cv_r2_std': cv_scores.std(),
        'test_r2': r2,
        'test_mae': mae,
        'test_rmse': rmse,
        'predictions': y_pred
    }
    
    print(f"  CV R² Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    print(f"  Test R² Score: {r2:.4f}")
    print(f"  Test MAE: {mae:.4f}")
    print(f"  Test RMSE: {rmse:.4f}")

print("\n=== MODEL COMPARISON ===")
comparison_df = pd.DataFrame({
    'Model': results.keys(),
    'CV R²': [results[name]['cv_r2_mean'] for name in results.keys()],
    'CV R² Std': [results[name]['cv_r2_std'] for name in results.keys()],
    'Test R²': [results[name]['test_r2'] for name in results.keys()],
    'Test MAE': [results[name]['test_mae'] for name in results.keys()],
    'Test RMSE': [results[name]['test_rmse'] for name in results.keys()]
})

comparison_df = comparison_df.sort_values('Test R²', ascending=False)
print(comparison_df.round(4))

# Find best model
best_model_name = comparison_df.iloc[0]['Model']
best_model = results[best_model_name]['model']
print(f"\nBest performing model: {best_model_name}")
print(f"Best model Test R²: {results[best_model_name]['test_r2']:.4f}")

## 8. Model Evaluation and Insights

In [ ]:
# Detailed model evaluation
print("=== MODEL EVALUATION AND INSIGHTS ===")

# Model comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# R² comparison
model_names = list(results.keys())
cv_scores = [results[name]['cv_r2_mean'] for name in model_names]
test_scores = [results[name]['test_r2'] for name in model_names]

x = np.arange(len(model_names))
width = 0.35

axes[0,0].bar(x - width/2, cv_scores, width, label='CV R²', alpha=0.8)
axes[0,0].bar(x + width/2, test_scores, width, label='Test R²', alpha=0.8)
axes[0,0].set_xlabel('Models')
axes[0,0].set_ylabel('R² Score')
axes[0,0].set_title('Model Performance Comparison')
axes[0,0].set_xticks(x)
axes[0,0].set_xticklabels(model_names, rotation=45, ha='right')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Prediction vs Actual for best model
best_predictions = results[best_model_name]['predictions']
axes[0,1].scatter(y_test, best_predictions, alpha=0.6, color='blue')
axes[0,1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0,1].set_xlabel('Actual G3')
axes[0,1].set_ylabel('Predicted G3')
axes[0,1].set_title(f'Predictions vs Actual - {best_model_name}')
axes[0,1].grid(True, alpha=0.3)

# Residuals plot
residuals = y_test - best_predictions
axes[1,0].scatter(best_predictions, residuals, alpha=0.6, color='green')
axes[1,0].axhline(y=0, color='red', linestyle='--')
axes[1,0].set_xlabel('Predicted G3')
axes[1,0].set_ylabel('Residuals')
axes[1,0].set_title(f'Residuals Plot - {best_model_name}')
axes[1,0].grid(True, alpha=0.3)

# Feature importance (if available)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False).head(15)
    
    axes[1,1].barh(range(len(feature_importance)), feature_importance['importance'])
    axes[1,1].set_yticks(range(len(feature_importance)))
    axes[1,1].set_yticklabels(feature_importance['feature'])
    axes[1,1].set_xlabel('Feature Importance')
    axes[1,1].set_title(f'Top 15 Feature Importances - {best_model_name}')
    axes[1,1].invert_yaxis()
elif hasattr(best_model, 'coef_'):
    # For linear models, show coefficient magnitudes
    coef_importance = pd.DataFrame({
        'feature': feature_cols,
        'coefficient': np.abs(best_model.coef_)
    }).sort_values('coefficient', ascending=False).head(15)
    
    axes[1,1].barh(range(len(coef_importance)), coef_importance['coefficient'])
    axes[1,1].set_yticks(range(len(coef_importance)))
    axes[1,1].set_yticklabels(coef_importance['feature'])
    axes[1,1].set_xlabel('|Coefficient|')
    axes[1,1].set_title(f'Top 15 Coefficient Magnitudes - {best_model_name}')
    axes[1,1].invert_yaxis()

plt.tight_layout()
plt.show()

# Print detailed insights
print("\n=== KEY INSIGHTS ===")
print(f"\n1. Model Performance:")
print(f"   - Best model: {best_model_name}")
print(f"   - Test R² Score: {results[best_model_name]['test_r2']:.4f}")
print(f"   - Test MAE: {results[best_model_name]['test_mae']:.4f}")
print(f"   - Test RMSE: {results[best_model_name]['test_rmse']:.4f}")

print(f"\n2. Feature Importance Analysis:")
if hasattr(best_model, 'feature_importances_'):
    print("   Top 5 most important features:")
    for i, (feature, importance) in enumerate(feature_importance.head(5).values):
        print(f"   {i+1}. {feature}: {importance:.4f}")
elif hasattr(best_model, 'coef_'):
    print("   Top 5 features with highest coefficient magnitude:")
    for i, (feature, coef) in enumerate(coef_importance.head(5).values):
        print(f"   {i+1}. {feature}: {coef:.4f}")

print(f"\n3. Alcohol Consumption Impact:")
alcohol_features = [col for col in feature_cols if 'alc' in col.lower()]
if alcohol_features:
    print("   Alcohol-related features in the model:")
    for feature in alcohol_features:
        if hasattr(best_model, 'feature_importances_'):
            idx = feature_cols.index(feature)
            importance = best_model.feature_importances_[idx]
            print(f"   - {feature}: {importance:.4f}")
        elif hasattr(best_model, 'coef_'):
            idx = feature_cols.index(feature)
            coef = best_model.coef_[idx]
            print(f"   - {feature}: {coef:.4f}")

print(f"\n4. Model Reliability:")
print(f"   - Cross-validation R² mean: {results[best_model_name]['cv_r2_mean']:.4f}")
print(f"   - Cross-validation R² std: {results[best_model_name]['cv_r2_std']:.4f}")
print(f"   - Generalization gap: {abs(results[best_model_name]['cv_r2_mean'] - results[best_model_name]['test_r2']):.4f}")

## 9. Conclusion and Recommendations

In [ ]:
# Final analysis and recommendations
print("=== FINAL ANALYSIS AND RECOMMENDATIONS ===")

# Calculate some key statistics for conclusions
high_alcohol_students = df[(df['Dalc'] >= 3) | (df['Walc'] >= 3)]
low_alcohol_students = df[(df['Dalc'] < 3) & (df['Walc'] < 3)]

print(f"\n1. DATASET SUMMARY:")
print(f"   - Total students analyzed: {len(df)}")
print(f"   - High alcohol consumers: {len(high_alcohol_students)} ({len(high_alcohol_students)/len(df)*100:.1f}%)")
print(f"   - Low alcohol consumers: {len(low_alcohol_students)} ({len(low_alcohol_students)/len(df)*100:.1f}%)")
print(f"   - Average final grade (G3): {df['G3'].mean():.2f}")
print(f"   - Weekend vs weekday drinking preference: {(df['Walc'] > df['Dalc']).sum()} students ({(df['Walc'] > df['Dalc']).mean()*100:.1f}%)")

print(f"\n2. KEY FINDINGS:")
print(f"   a) Academic Performance Impact:")
print(f"      - High alcohol consumers average G3: {high_alcohol_students['G3'].mean():.2f}")
print(f"      - Low alcohol consumers average G3: {low_alcohol_students['G3'].mean():.2f}")
print(f"      - Performance difference: {low_alcohol_students['G3'].mean() - high_alcohol_students['G3'].mean():.2f} points")

print(f"   b) Alcohol Consumption Patterns:")
print(f"      - Average weekday consumption: {df['Dalc'].mean():.2f}/5")
print(f"      - Average weekend consumption: {df['Walc'].mean():.2f}/5")
print(f"      - Correlation between Dalc and Walc: {df['Dalc'].corr(df['Walc']):.3f}")

print(f"   c) Demographic Insights:")
male_avg = df[df['sex'] == 'M'][['Dalc', 'Walc']].mean()
female_avg = df[df['sex'] == 'F'][['Dalc', 'Walc']].mean()
print(f"      - Male students average alcohol: Dalc={male_avg['Dalc']:.2f}, Walc={male_avg['Walc']:.2f}")
print(f"      - Female students average alcohol: Dalc={female_avg['Dalc']:.2f}, Walc={female_avg['Walc']:.2f}")

print(f"   d) Predictive Modeling:")
print(f"      - Best model: {best_model_name}")
print(f"      - Prediction accuracy (R²): {results[best_model_name]['test_r2']:.4f}")
print(f"      - Average prediction error (MAE): {results[best_model_name]['test_mae']:.2f} points")

print(f"\n3. RECOMMENDATIONS:")
print(f"   a) For Educational Institutions:")
print(f"      - Monitor students with high weekend alcohol consumption (Walc ≥ 3)")
print(f"      - Implement intervention programs for students showing declining academic performance")
print(f"      - Focus on students with multiple risk factors (high alcohol + low study time + high absences)")

print(f"   b) For Prevention Programs:")
print(f"      - Target male students who show higher alcohol consumption rates")
print(f"      - Address the strong correlation between going out frequency and alcohol consumption")
print(f"      - Develop programs that promote healthy social activities")

print(f"   c) For Further Research:")
print(f"      - Investigate the temporal relationship between alcohol consumption and academic decline")
print(f"      - Study the effectiveness of intervention programs on reducing alcohol-related academic issues")
print(f"      - Explore protective factors that help students maintain good grades despite alcohol consumption")

print(f"\n4. LIMITATIONS:")
print(f"   - Self-reported alcohol consumption data may be subject to bias")
print(f"   - Cross-sectional data limits causal inference")
print(f"   - Sample limited to Portuguese students, may not generalize to other populations")
print(f"   - Alcohol consumption scale (1-5) may not capture fine-grained differences")

print(f"\n5. MODEL DEPLOYMENT RECOMMENDATIONS:")
print(f"   - Use the {best_model_name} model for predicting at-risk students")
print(f"   - Implement regular model retraining with new data")
print(f"   - Set up alerts for students with predicted G3 < 10 (failing grade)")
print(f"   - Combine predictions with human expert judgment for intervention decisions")

print("\n" + "="*80)
print("ANALYSIS COMPLETED SUCCESSFULLY!")
print("This comprehensive analysis provides insights into student alcohol consumption")
print("patterns and their relationship with academic performance.")
print("="*80)